In [1]:
!pip install gupload

     |████████████████████████████████| 81kB 2.8MB/s 
     |████████████████████████████████| 61kB 3.7MB/s 
ERROR: earthengine-api 0.1.266 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.7.10 which is incompatible.
  Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import gc
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import LabelEncoder
import sys

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Sales train
id_dataset = '1qFeVYGuUQu6KN27Ezv6GTx_cAnAJaz95'
df = drive.CreateFile({'id':id_dataset})   
df.GetContentFile('sales_train.csv')
# Items
id_dataset = '14uHM25XynWK3r6jCwfL2F572iyV3S2kg'
items = drive.CreateFile({'id':id_dataset})   
items.GetContentFile('items.csv')
# Items Categories
id_dataset = '1jnW89GPUvQMVYATkplS01FSACDU1i0Tn'
items_cat = drive.CreateFile({'id':id_dataset})   
items_cat.GetContentFile('item_categories_translated.csv')
# Shops
id_dataset = '1rI6KyOnCIgU5xpQQxLY8F5g8xdc1Ix3P'
shops = drive.CreateFile({'id':id_dataset})   
shops.GetContentFile('shops_translated.csv')
# Sales test
id_dataset = '1CFlPRu89rIV1qOrdQLYRiSZXJd-1bK1V'
test = drive.CreateFile({'id':id_dataset})   
test.GetContentFile('test.csv')

# Read all files
sales            = pd.read_csv('sales_train.csv')
items            = pd.read_csv('items.csv')
item_categories  = pd.read_csv('item_categories_translated.csv')
shops            = pd.read_csv('shops_translated.csv')
test             = pd.read_csv('test.csv')

In [5]:
def downcast(df):

    '''
    Changes column types in the dataframe:  
    'bool'    type to 'int8'
    `float64` type to 'float16'
    `int64`   type to 'int16'
    '''

    # Select columns to downcast
    float_cols = [x for x in df if df[x].dtype == 'float64'
    int_cols   = [x for x in df if df[x].dtype == 'int64']
    bool_cols  = [x for x in df if df[x].dtype == 'bool']
    # Downcast
    df[bool_cols] = df[bool_cols].astype(np.int8)
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int16)
    
    return df

### Shop's dataframe preprocessing



In [6]:
shops['city'] = shops['shop_name'].apply(lambda x: x.replace('!','').split()[0])

In [7]:
# Encoding our city feature
label_encoder = LabelEncoder()
shops['city'] = label_encoder.fit_transform(shops['city'])

### Item's dataframe preprocessing

In [8]:
# Create a feature with the 1st-ever-sale date for every item
items['first_sale_date'] = sales.groupby('item_id').agg({'date_block_num': 'min'})['date_block_num']
items['first_sale_date'] = items['first_sale_date'].fillna(34)

### Item_categories' dataframe preprocessing

In [9]:
# Create a major feature
item_categories['major'] = item_categories['item_category_name'].apply(lambda x: x.split()[0])  

In [10]:
# Replace with 'etc' if category count is less than 5
item_categories['major'] = item_categories['major'].apply(lambda x: x if len(item_categories[item_categories['major']==x]) >= 5 else 'other')

In [11]:
# Encoding our major variable
label_encoder = LabelEncoder()
item_categories['major'] = label_encoder.fit_transform(item_categories['major'])

### Working with sales dataframe

In [12]:
# Outliers removal
sales = sales[(sales['item_price'] > 0) & (sales['item_price'] < 50000)]
sales = sales[(sales['item_cnt_day'] > 0) & (sales['item_cnt_day'] < 1000)]

In [13]:
# Some shops are duplicated w/distinct shop_id and shop_name
print(f'> {shops.iloc[0,0]} (id: {shops.iloc[0,1]})         <--->  \
{shops.iloc[57,0]} (id: {shops.iloc[57,1]})\n\
> {shops.iloc[1,0]} (id: {shops.iloc[1,1]}) <--->  \
{shops.iloc[58,0]} (id: {shops.iloc[58,1]})\n\
> {shops.iloc[10,0]} (id: {shops.iloc[10,1]})             <--->  \
{shops.iloc[11,0]} (id: {shops.iloc[11,1]})\n\
> {shops.iloc[39,0]} (id: {shops.iloc[39,1]})       <--->  \
{shops.iloc[40,0]} (id: {shops.iloc[40,1]})\n')

> 0 (id: ! Yakutsk Ordzhonikidze, 56 fran)         <--->  57 (id: Yakutsk Ordzhonikidze, 56)
> 1 (id: ! Yakutsk shopping center "Central" Fran) <--->  58 (id: Yakutsk shopping center "Central")
> 10 (id: Zhukovsky st. Chkalova 39m?)             <--->  11 (id: Zhukovsky st. Chkalova 39m²)
> 39 (id: Rostnone TRK "Megacentr Horizont")       <--->  40 (id: Rostov on the Don TRK "Megcenter Horizon" island)



In [14]:
# We correct this cases
sales['shop_id'] = sales['shop_id'].replace({0: 57, 1: 58, 11: 10, 40: 39})
test['shop_id'] = test['shop_id'].replace({0: 57, 1: 58, 11: 10, 40: 39})

In [15]:
# Some of the shops that are in the sales dataframe are not in the Test set. 
# We Leak to improve predictions
shops_on_test = test['shop_id'].unique()
sales = sales[sales['shop_id'].isin(shops_on_test)]

### Preprocessing

Test set is a result of all posible combinations of shops + items for a given month (a cartesian product), we'll mimic that approach in our train set

In [16]:
index_cols = ['date_block_num','shop_id', 'item_id']

# We get the cartesian product for every month in the train set
combinations = [] 
for block_num in sales['date_block_num'].unique():
  cur_shops     = sales[sales['date_block_num']==block_num]['shop_id'].unique()
  cur_items     = sales[sales['date_block_num']==block_num]['item_id'].unique()
  combinations.append(np.array(list(product([block_num],cur_shops, cur_items))))

In [17]:
# Turn the combinations into a pandas dataframe
combinations = pd.DataFrame(np.vstack(combinations), columns = index_cols)

# Generate aggregated features on item_cnt_day and item_price
gb = sales.groupby(index_cols).agg({'item_cnt_day':'sum', 'item_price':'mean'})
gb = gb.reset_index()
gb.rename(columns={'item_cnt_day':'item_cnt_month',\
                   'item_price':'item_price_mean'}, inplace=True)

# Join the data to the combinations
df = pd.merge(combinations,gb,how='left',on=index_cols)

# Generate a feature for the number of items sold
gb = sales.groupby(index_cols).agg({'item_cnt_day': 'count'})
gb = gb.reset_index()
gb = gb.rename(columns={'item_cnt_day': 'item_count'})

df = pd.merge(df, gb, on=index_cols, how='left')


del combinations, gb, sales
gc.collect();

### Concatenate test data, Merge remaining data

In [18]:
# Lets append to the DF the Test data
test['date_block_num'] = 34
df = pd.concat([df, test.drop('ID',axis=1)], ignore_index=True,\
               keys=index_cols)
df.fillna(0,inplace=True)

In [19]:
shops.drop(['shop_name','Unnamed: 0'],axis=1,inplace=True)
item_categories.drop(['item_category_name','Unnamed: 0'],axis=1,inplace=True)
items.drop(['item_name'],axis=1,inplace=True)

# get the final merged df
df = df.merge(shops, on='shop_id', how='left')
df = df.merge(items, on='item_id', how='left')
df = df.merge(item_categories, on='item_category_id', how='left')


df = downcast(df)

del shops, items, item_categories
gc.collect();

## Feature Engineering

### Mean Encoding

In [20]:
def mean_encodings(df, features):

  '''
  Create mean encodings on item_cnt_month based on the idx_features passed
  '''

  temporal = df.groupby(features).agg({'item_cnt_month': 'mean'})
  temporal.reset_index(inplace=True)
  names = features + ['mean_sales']
  temporal.rename(columns={'item_cnt_month':'_'.join(names)}, inplace=True)
  
  #Mege with the dataset
  df = pd.merge(df, temporal, on=features, how='left')
  df = downcast(df)
  
  del temporal
  gc.collect();

  return df

In [21]:
df = mean_encodings(df,['date_block_num','item_id'])
df = mean_encodings(df,['date_block_num','item_id','city'])
df = mean_encodings(df,['date_block_num','shop_id','item_category_id'])

### Create Lag Features

In [22]:
def lag_features(df, idx_features,lag_feature):

  '''
  Create lag features for -1,-2,-3 months on the idx_features passed
  '''

  df_temp = df[idx_features + [lag_feature]].copy() 

  # Create lag features
  for month in range(1, 4):
      lag_feature_name = lag_feature +'_lag' + str(month)
      df_temp.columns = idx_features + [lag_feature_name]

      # Add 1 to date_block_num to merge df_temp with the passed df
      df_temp['date_block_num'] += month

      # Merge df with df_temp based on idx_feature
      df = df.merge(df_temp.drop_duplicates(),on=idx_features,how='left')

      # Fillna with 0 
      df[lag_feature_name] = df[lag_feature_name].fillna(0)

  df = downcast(df)


  del df_temp
  gc.collect()
  
  return df

In [23]:
#Features to lag
features = ['item_price_mean','item_cnt_month','item_count', 'date_block_num_item_id_mean_sales',\
                                                                  'date_block_num_item_id_city_mean_sales']

for feat in features:
  df = lag_features(df=df, 
        idx_features=['date_block_num', 'shop_id', 'item_id'] ,
        lag_feature=feat)

df = lag_features(df=df, 
      idx_features=['date_block_num', 'shop_id', 'item_category_id'],
      lag_feature='date_block_num_shop_id_item_category_id_mean_sales')

In [24]:
# Remove data from the first 2 months as they will have no lag data
df = df.drop(df[df['date_block_num'] < 3].index)

### Final Feature Engineering

In [25]:
# Generate mean from the last 3 months item_cnt_month
df['3_Months_Mean'] = df[['item_cnt_month_lag1', 'item_cnt_month_lag2', 'item_cnt_month_lag3']].mean(axis=1)

As we saw in the EDA file, the distribution from the Train set shows that the item_cnt_day is concentrated arround...
So we are going to clip this values from 0 up to 20 (increasing our prediction score +10%)

In [26]:
clip = ['item_cnt_month', 'item_cnt_month_lag1',
       'item_cnt_month_lag2', 'item_cnt_month_lag3', 'item_count_lag1',
       'item_count_lag2', 'item_count_lag3',
       'date_block_num_item_id_mean_sales_lag1',
       'date_block_num_item_id_mean_sales_lag2',
       'date_block_num_item_id_mean_sales_lag3',
       'date_block_num_item_id_city_mean_sales_lag1',
       'date_block_num_item_id_city_mean_sales_lag2',
       'date_block_num_item_id_city_mean_sales_lag3',
       'date_block_num_shop_id_item_category_id_mean_sales_lag1',
       'date_block_num_shop_id_item_category_id_mean_sales_lag2',
       'date_block_num_shop_id_item_category_id_mean_sales_lag3','3_Months_Mean']

for col in clip:
  df[col] = df[col].clip(0,20)

In [27]:
# Generate percentage evolution from month to month
df['evolution_1'] = df['item_cnt_month_lag1']/df['item_cnt_month_lag2']
df['evolution_1'] = df['evolution_1'].replace([np.inf, -np.inf],np.nan).fillna(0)
df['evolution_2'] = df['item_cnt_month_lag2']/df['item_cnt_month_lag3']
df['evolution_2'] = df['evolution_2'].replace([np.inf, -np.inf],np.nan).fillna(0)

In [28]:
# Generate a binary future for the first_month of every item
df['first_month'] = df['first_sale_date'] == df['date_block_num']

In [29]:
# Generate a permanence feature
df['permanence'] = df['date_block_num'] - df['first_sale_date']

In [30]:
# Generate the n° of month of every instance
df['month'] = df['date_block_num']%12

In [31]:
# Drop the columns from that bring data from the current month and make final
# dataframe preprocessing
toDrop = ['item_price_mean', 'item_count',\
          'date_block_num_shop_id_item_category_id_mean_sales',\
          'date_block_num_item_id_city_mean_sales',\
          'date_block_num_item_id_mean_sales','first_sale_date']

df.drop(toDrop, axis=1, inplace=True)
df = downcast(df)
df.rename(columns={'item_cnt_month':'target'}, inplace=True)

In [32]:
df.to_pickle('df_processed.pkl')
!gupload --to '1oBAu3xn2ZCcWOAI6ZnAcDdUSAMyTllai' df_processed.pkl

Uploading file: df.pkl
tcmalloc: large alloc 1140236288 bytes == 0x55d113c04000 @  0x7fe4fd5f11e7 0x55d0cd374e68 0x55d0cd38bea6 0x55d0cd38bb88 0x55d0cd3942c0 0x55d0cd39a721 0x55d0cd39d695 0x55d0cd3403c3 0x55d0cd342c7c 0x55d0cd433fed 0x55d0cd3b6988 0x55d0cd3b14ae 0x55d0cd3443ea 0x55d0cd3b260e 0x55d0cd3b17ad 0x55d0cd3443ea 0x55d0cd3b332a 0x55d0cd3b17ad 0x55d0cd3443ea 0x55d0cd3b23b5 0x55d0cd3b17ad 0x55d0cd283e2c 0x55d0cd3b3bb5 0x55d0cd3b14ae 0x55d0cd344c9f 0x55d0cd344ea1 0x55d0cd3b3bb5 0x55d0cd34430a 0x55d0cd3b260e 0x55d0cd3b14ae 0x55d0cd344a81
tcmalloc: large alloc 1710350336 bytes == 0x55d157b6e000 @  0x7fe4fd5f11e7 0x55d0cd374e68 0x55d0cd33f637 0x55d0cd455361 0x55d0cd34045d 0x55d0cd39e2d2 0x55d0cd39d443 0x55d0cd342c7c 0x55d0cd433fed 0x55d0cd3b6988 0x55d0cd34430a 0x55d0cd3b23b5 0x55d0cd3b17ad 0x55d0cd3443ea 0x55d0cd3b23b5 0x55d0cd34430a 0x55d0cd3b260e 0x55d0cd34430a 0x55d0cd3b260e 0x55d0cd3b14ae 0x55d0cd3443ea 0x55d0cd3b332a 0x55d0cd34430a 0x55d0cd3b23b5 0x55d0cd34430a 0x55d0cd3b260e 0x